<a href="https://www.kaggle.com/code/tamswgner/yolov5-vehicle-detection?scriptVersionId=152632482" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#YOLOv5 install
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

#Necessary imports
import torch
import utils
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
import os
import glob
import shutil
import numpy as np
import yaml

In [ ]:
#YOLO installation check
display = utils.notebook_init()
%cd ..

In [ ]:
#Remove old dataset from the env
%rm -r /kaggle/working/data
if not os.path.exists('data'):
    for folder in ['images', 'labels']:
        for split in ['train','val','test']:
            os.makedirs(f'data/{folder}/{split}')

In [ ]:
#Additional logging tools
%load_ext tensorboard
%tensorboard --logdir runs/train

In [ ]:
#Select the sub-dataset with the folders
dataset = 'sec3'

In [ ]:
#Bonding box drawer function
def drawBbox(imgPath):
    annoPath = imgPath.replace('.png', '.txt')
    print(annoPath)
    img = Image.open(imgPath)
    draw = ImageDraw.Draw(img)

    with open(annoPath,'r') as file:
        for line in file.readlines():
            anno,x_cen,y_cen,width,height = line.split(' ')

            x_cen = float(x_cen)
            y_cen = float(y_cen)
            width = float(width)
            height = float(height)

            imgWidth, imgHeight = img.size
            x1 = (x_cen-width/2) * imgWidth
            x2 = (x_cen+width/2) * imgWidth
            y1 = (y_cen-height/2) * imgHeight
            y2 = (y_cen+height/2) * imgHeight

            draw.rectangle((x1,y1,x2,y2),outline=(0,255,0), width=5)

    plt.imshow(img)
    plt.show()

In [ ]:
#Check wether the dataset is correct or not
drawBbox(f'/kaggle/input/traffic-images-captured-from-uavs/dataset/{dataset}/{dataset}_frame_000101.png')

In [ ]:
#Function to read all image names from the dataset
def get_filenames(folder):
    filenames = set()
    
    for path in glob.glob(os.path.join(folder, '*.png')):
        # Extract the filename
        filename = os.path.split(path)[-1]        
        filenames.add(filename)

    return filenames


In [ ]:
#Load dataset into list
section = get_filenames(f'/kaggle/input/traffic-images-captured-from-uavs/dataset/{dataset}')

In [ ]:
#Convert dataset into numpy array for test,train,validation split
np_section = np.array(list(section))

# Use the same random seed for reproducability
np.random.seed(69)
np.random.shuffle(np_section)

#Check the amount of images
print(np_section.size)

In [ ]:
#Split dataset into test,train,validation sections
def split_dataset(label, image_names, train_size, val_size):
    for i, image_name in enumerate(image_names):
        label_name = image_name.replace('.png', '.txt')
        
        if i < train_size:
            split = 'train'
        elif i < train_size + val_size:
            split = 'val'
        else:
            split = 'test'
        
        source_image_path = f'/kaggle/input/traffic-images-captured-from-uavs/dataset/{label}/{image_name}'
        source_label_path = f'/kaggle/input/traffic-images-captured-from-uavs/dataset/{label}/{label_name}'
        #print(source_label_path)

        target_image_folder = f'data/images/{split}'
        target_label_folder = f'data/labels/{split}'

        shutil.copy(source_image_path, target_image_folder)
        shutil.copy(source_label_path, target_label_folder)

In [ ]:
split_dataset(dataset, np_section, train_size=1000, val_size=100)

In [ ]:
# Define the data for the YOLO YAML file as a dictionary
data = {
    'train': '/kaggle/working/data/images/train',
    'val': '/kaggle/working/data/images/val',
    'test': '/kaggle/working/data/images/test',
    'nc': 2,
    'names': ['car', 'motorbike']
}

# Convert the dictionary to a YAML string
yaml_str = yaml.dump(data, sort_keys=False)

filename = 'dataset.yaml'

with open(filename, 'w') as file:
    file.write(yaml_str)

print(f'YAML file created: {filename}')

In [ ]:
#Fine tuning a pretrained YOLO network, and freezing the backbone layers
%cd /kaggle/working/
!python yolov5/train.py --data dataset.yaml --weights yolov5l.pt --epochs 50 --batch 64 --freeze 10

In [ ]:
#Load the folder with the trained networks
%cd /kaggle/working/yolov5/runs/train/exp6/weights

In [ ]:
# Load trained network
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/kaggle/working/yolov5/runs/train/exp6/weights/best.pt', force_reload=True)

# Load test image
img = "/kaggle/input/traffic-images-captured-from-uavs/dataset/sec6/sec6_frame_000239.png"

# Predict
results = model(img)
print(results.show())  

In [ ]:
#Create a downloadable zip from the test process
%cd /kaggle/working/
%zip -r file.zip /kaggle/working/yolov5/runs